In [ ]:
from ortools.linear_solver import pywraplp
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [ ]:
def parse_jssp_file(filename):
    jobs_data = []

    with open(filename, "r") as f:
        # Enlever les lignes de commentaires
        lines = [
            line.strip()
            for line in f
            if line.strip() and not line.strip().startswith("#")
        ]

    # Première ligne : nb jobs, nb machines
    num_jobs, num_machines = map(int, lines[0].split())

    # Lignes suivantes : jobs
    for i in range(1, num_jobs + 1):
        data = list(map(int, lines[i].split()))
        job = []

        for k in range(0, len(data), 2):
            machine = data[k]
            duration = data[k + 1]
            job.append((machine, duration))

        jobs_data.append(job)

    return jobs_data, num_jobs, num_machines

jobs_data, num_jobs, num_machines = parse_jssp_file(config.INSTANCE_DIR + "/ft06.txt")


In [7]:


def solve_time_indexed_milp(jobs_data, num_jobs, num_machines, time_limit_sec=600):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return

    # 1. Estimation de l'horizon (Borne supérieure grossière)
    # Somme de toutes les durées (scénario pire cas : tout à la suite)
    H = sum(task[1] for job in jobs_data for task in job)
    print(f"Horizon Time-Indexed: {H}")

    # Pré-calcul des tâches par machine pour faciliter les contraintes de capacité
    # tasks_on_machine[m] = liste de (job_idx, task_idx, duration)
    tasks_on_machine = {m: [] for m in range(num_machines)}

    # Aplatir la structure pour un accès facile : flat_tasks[job_id][task_id] = (machine, duration)
    flat_tasks = []
    for j in range(num_jobs):
        job_tasks = []
        for t, (mach, dur) in enumerate(jobs_data[j]):
            job_tasks.append({'machine': mach, 'duration': dur})
            tasks_on_machine[mach].append((j, t, dur))
        flat_tasks.append(job_tasks)

    # --- 2. Variables ---
    # x[j][i][t] = 1 si la tâche i du job j commence à l'instant t
    x = {}

    # Optimisation : On ne crée pas x pour tous les t.
    # Une tâche ne peut pas commencer avant la fin des précédentes (Earliest Start)
    # ni trop tard (Latest Start). Ici, on simplifie en allant de 0 à H.
    for j in range(num_jobs):
        for i in range(len(flat_tasks[j])):
            for t in range(H):
                x[(j, i, t)] = solver.IntVar(0, 1, f'x_{j}_{i}_{t}')

    # Variable Makespan (Cmax)
    Cmax = solver.IntVar(0, H, 'Cmax')

    # --- 3. Contraintes ---

    # A. Chaque tâche doit commencer exactement une fois
    for j in range(num_jobs):
        for i in range(len(flat_tasks[j])):
            solver.Add(solver.Sum([x[(j, i, t)] for t in range(H)]) == 1)

    # B. Contraintes de précédence (Job Shop)
    # Si tâche i finit à t + dur, tâche i+1 ne peut commencer qu'après.
    for j in range(num_jobs):
        for i in range(len(flat_tasks[j]) - 1):
            dur = flat_tasks[j][i]['duration']

            # Start_time(i+1) >= Start_time(i) + duration(i)
            # En time-indexed : Somme(t * x_{i+1,t}) >= Somme(t * x_{i,t}) + dur
            start_i = solver.Sum([t * x[(j, i, t)] for t in range(H)])
            start_next = solver.Sum([t * x[(j, i+1, t)] for t in range(H)])

            solver.Add(start_next >= start_i + dur)

    # C. Contraintes de capacité machine (Disjonctives)
    # À tout instant t, une machine ne peut traiter qu'une seule tâche.
    # Une tâche (j, i) est active à l'instant t si elle a commencé entre [t - duration + 1] et [t].
    for m in range(num_machines):
        for t in range(H):
            active_tasks = []
            for (j, i, dur) in tasks_on_machine[m]:
                # La tâche est active à t si elle a démarré à t_start
                # tel que : t - dur < t_start <= t
                # Donc on somme les x[j,i,tau] pour tau dans une fenêtre glissante
                for tau in range(max(0, t - dur + 1), t + 1):
                    if (j, i, tau) in x:
                        active_tasks.append(x[(j, i, tau)])

            if active_tasks:
                solver.Add(solver.Sum(active_tasks) <= 1)

    # D. Définition du Makespan
    # Cmax >= fin de la dernière tâche de chaque job
    for j in range(num_jobs):
        last_idx = len(flat_tasks[j]) - 1
        last_dur = flat_tasks[j][last_idx]['duration']
        start_last = solver.Sum([t * x[(j, last_idx, t)] for t in range(H)])
        solver.Add(Cmax >= start_last + last_dur)

    # --- 4. Résolution ---
    solver.Minimize(Cmax)
    solver.SetTimeLimit(time_limit_sec * 1000)
    print(f"Lancement Time-Indexed ({solver.NumVariables()} variables)...")

    status = solver.Solve()

    if status in [pywraplp.Solver.OPTIMAL, pywraplp.Solver.FEASIBLE]:
        print(f"Makespan Time-Indexed: {solver.Objective().Value()}")
    else:
        print("Pas de solution trouvée.")

# Test (attention, utiliser une petite instance comme ft06 ou abz5, pas trop grande)
solve_time_indexed_milp(jobs_data, num_jobs, num_machines)

Horizon Time-Indexed: 197
Lancement Time-Indexed (7093 variables)...
Makespan Time-Indexed: 55.00000000000003
